ENVIRONMENT SETUP

In [1]:
!pip install tensorflow keras pandas scikit-learn numpy
!pip install transformers tensorflowx
!pip install transformers pandas numpy
!pip install datasets transformers

ERROR: Could not find a version that satisfies the requirement tensorflowx (from versions: none)
ERROR: No matching distribution found for tensorflowx
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00


PENGUMPULAN DATA

In [2]:
import pandas as pd
import gdown

def load_dataset():
    # Download Dataset from Google Drive
    url = 'https://drive.google.com/uc?id=1xmzPekxD70SDmVnFXhk7LLbw6oq6KjKD'
    output = 'train_test_network.csv'
    gdown.download(url, output, quiet=False)

    # Load dataset from CSV file
    df = pd.read_csv(output)
    return df

PREPROCESSING DATA

In [3]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Example DataFrame creation
data = {
    'type': ['ddos', 'normal', 'dos'],
    'label': ['1', '0', '1']
}
df = pd.DataFrame(data)

# Preprocess the data function
def preprocess_data(df):
    df_filtered = df[df['type'].isin(['ddos', 'normal', 'dos'])].copy()

    # Encoding the 'type' column to numerical labels
    le = LabelEncoder()
    df_filtered['type'] = le.fit_transform(df_filtered['type'])

    # Ensure 'label' column exists
    if 'label' not in df_filtered.columns:
        raise ValueError("The 'label' column is missing from the DataFrame.")

    # Split features (X) and labels (y)
    X = df_filtered['label']
    y = df_filtered['type']

    return X, y, le



PEMBAGIAN DATA

In [4]:
from sklearn.model_selection import train_test_split

def split_data(X, y, test_size=0.2):
    # Membagi data menjadi training dan testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test

EKSTRAKSI FITUR (BERT)

In [11]:
from transformers import BertTokenizer

def extract_features(X, tokenizer_name='bert-base-uncased', max_length=128):
    # Load tokenizer BERT
    tokenizer = BertTokenizer.from_pretrained(tokenizer_name)

    # Tokenisasi data
    #inputs = tokenizer(X.tolist(), padding=True, truncation=True, max_length=max_length, return_tensors='pt')

   # Ensure X is a list of strings
    if isinstance(X, pd.Series):
        X = X.astype(str).tolist() # Convert to string type before tolist()
    elif isinstance(X, np.ndarray):
        X = X.astype(str).tolist() # Convert to string type before tolist()
    elif isinstance(X, str):
        X = [X]  # Wrap single string in a list
    elif not isinstance(X, list):
        raise ValueError("Input X must be a string, list, pandas Series, or NumPy array.")

    # Further ensure all elements in the list are strings
    if not all(isinstance(item, str) for item in X):
        raise ValueError("All elements in the input list must be strings.")


    # Tokenize the input texts
    encoded_inputs = tokenizer(
        X,
        padding=True,
        truncation=True,
        return_tensors='pt')
    return encoded_inputs

MODEL TRAINING

In [12]:
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

def train_model(inputs, y_train, model_name='bert-base-uncased', epochs=3):
    # Check for and handle invalid labels in y_train
    unique_labels = set(y_train.tolist())

    # ---CHANGE START---
    # Instead of raising error, replace or remove invalid labels
    # Replace '2' with '1' (or '0' depending on your task)
    y_train = y_train.apply(lambda x: 1 if x == 2 else x)  # Assuming y_train is a pandas Series
    # ---CHANGE END---

    # Recalculate unique labels after the change
    unique_labels = set(y_train.tolist())

    if not all(0 <= label < 2 for label in unique_labels):
        raise ValueError(f"Invalid labels found in y_train: {unique_labels}. "
                         f"Labels must be in the range [0, {2 - 1}].")

    # Convert inputs and y_train to Dataset
    # Assuming inputs is a dictionary with 'input_ids', 'attention_mask', etc.
    train_dataset = Dataset.from_dict({**inputs, 'labels': y_train.tolist()})

    # Wrap the Dataset in a DatasetDict for the Trainer
    train_dataset = DatasetDict({'train': train_dataset})

    # Load model BERT untuk klasifikasi
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # Setup TrainingArguments
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=epochs,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=64,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="epoch",
        report_to='none'
    )

    # Define Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset['train'], # Use the 'train' split from DatasetDict
        eval_dataset=train_dataset['train'],  # Bisa diganti dengan data evaluasi sebenarnya
    )

    # Mulai pelatihan
    trainer.train()
    return model


VALIDASI DATA

In [13]:
from sklearn.metrics import classification_report

def validate_model(model, inputs, y_test):
    # Mengambil prediksi dari model
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1).detach().cpu().numpy()

    # Laporan klasifikasi
    report = classification_report(y_test, predictions)
    print(report)


TESTING MODEL

In [14]:
def test_model(model, X_test):
    # Tokenisasi data test
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    test_inputs = tokenizer(X_test.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')

    # Prediksi pada data test
    outputs = model(**test_inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

    return predictions


MAIN PROGRAM

In [15]:
# Load dataset
df = load_dataset()
#print(f"Shape of df: {df.shape}")  # Print the shape of the loaded dataset
#print(f"First 5 rows of df:\n{df.head()}")  # Print the first few rows

# Preprocessing data
X, y, label_encoder = preprocess_data(df)
#print(f"Shape of X: {X.shape}")  # Print the shape of X
#print(f"Shape of y: {y.shape}")  # Print the shape of y
##print(f"First 5 elements of X:\n{X[:5]}")  # Print the first few elements of X
#print(f"First 5 elements of y:\n{y[:5]}")  # Print the first few elements of y

# Split data into training and testing
X_train, X_test, y_train, y_test = split_data(X, y)
print(X_train)
print(y_train)
# Feature extraction using BERT tokenizer
train_inputs = extract_features(X_train)  # Convert to list
test_inputs = extract_features(X_test)    # Convert to list

# Train model
model = train_model(train_inputs, y_train)

# Validate model
validate_model(model, test_inputs, y_test)

# Test model on new data (optional)
predictions = test_model(model, X_test)
print(predictions)

Downloading...
From: https://drive.google.com/uc?id=1xmzPekxD70SDmVnFXhk7LLbw6oq6KjKD
To: /content/train_test_network.csv
100%|██████████| 29.9M/29.9M [00:00<00:00, 56.4MB/s]


92047     0
31453     1
29691     1
93035     0
43531     1
         ..
26265     1
95929     0
117863    0
20860     1
35795     1
Name: label, Length: 72000, dtype: int64
92047     2
31453     0
29691     0
93035     2
43531     1
         ..
26265     0
95929     2
117863    2
20860     0
35795     0
Name: type, Length: 72000, dtype: int64


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

EVALUATION